In [1]:
from __future__ import print_function

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf

/home/devendra/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
with tf.Session() as sess:
    for i in range(2):
        batch_x , batch_y = mnist.train.next_batch(128)
        print(batch_x.shape)
        print(batch_y.shape)

(128, 784)
(128, 10)
(128, 784)
(128, 10)


## 1 - create placeholders

In [3]:
def create_placeholders(n_x , n_y):
    X = tf.placeholder(tf.float32 , (n_x,None))
    Y = tf.placeholder(tf.float32 , (n_y,None))
    return X,Y

In [4]:
X,Y = create_placeholders(784,10)
print("x = "+str(X))
print("y = "+str(Y))

x = Tensor("Placeholder:0", shape=(784, ?), dtype=float32)
y = Tensor("Placeholder_1:0", shape=(10, ?), dtype=float32)


## 2 - Initializing the parameters

In [5]:
def initialize_parameters():
    tf.set_random_seed(1)
    xavier_initializer = tf.contrib.layers.xavier_initializer(seed=1)
    zeros_initializer = tf.zeros_initializer()
    W1 = tf.get_variable("W1",[256,784],initializer=xavier_initializer)
    b1 = tf.get_variable("b1",[256,1],initializer=zeros_initializer)
    
    W2 = tf.get_variable("W2",[256,256],initializer=xavier_initializer)
    b2 = tf.get_variable("b2",[256,1],initializer=zeros_initializer)
    
    W3 = tf.get_variable("W3",[10,256],initializer=xavier_initializer)
    b3 = tf.get_variable("b3",[10,1],initializer=zeros_initializer)
    
    parameters = {
        "W1":W1,
        "b1":b1,
        "W2":W2,
        "b2":b2,
        "W3":W3,
        "b3":b3
    }
    return parameters

In [6]:
tf.reset_default_graph()
with tf.Session() as sess:
    parameters = initialize_parameters()
    print("W1 = "+str(parameters["W1"]))
    print("b1 = "+str(parameters["b1"]))
    print("W2 = "+str(parameters["W2"]))
    print("b2 = "+str(parameters["b2"]))
    print("W3 = "+str(parameters["W3"]))
    print("b3 = "+str(parameters["b3"]))

W1 = <tf.Variable 'W1:0' shape=(256, 784) dtype=float32_ref>
b1 = <tf.Variable 'b1:0' shape=(256, 1) dtype=float32_ref>
W2 = <tf.Variable 'W2:0' shape=(256, 256) dtype=float32_ref>
b2 = <tf.Variable 'b2:0' shape=(256, 1) dtype=float32_ref>
W3 = <tf.Variable 'W3:0' shape=(10, 256) dtype=float32_ref>
b3 = <tf.Variable 'b3:0' shape=(10, 1) dtype=float32_ref>


## 3 - Forward propagation in tensorflow

In [7]:
def forward_propagation(X,parameters):
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]
    
    Z1 = tf.add(tf.matmul(W1,X),b1)
    Z2 = tf.add(tf.matmul(W2,Z1),b2)
    Z3 = tf.add(tf.matmul(W3,Z2),b3)
    
    return Z3

In [8]:
tf.reset_default_graph()
with tf.Session() as sess:
    X , Y = create_placeholders(784,10)
    parameters = initialize_parameters()
    Z3 = forward_propagation(X,parameters)
    print("Z3 = " + str(Z3))

Z3 = Tensor("Add_2:0", shape=(10, ?), dtype=float32)


## 4  - Compute cost

In [9]:
def compute_cost(Z3,Y):
    logits = tf.transpose(Z3)
    labels = tf.transpose(Y)
    
    cost  = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=labels))
    
    return cost

In [10]:
tf.reset_default_graph()
with tf.Session() as sess:
    X,Y = create_placeholders(784,10)
    parameters = initialize_parameters()
    Z3 = forward_propagation(X,parameters)
    cost = compute_cost(Z3,Y)
    print("cost = "+str(cost))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

cost = Tensor("Mean:0", shape=(), dtype=float32)


## 5 - Backward propagation

For instance, for gradient descent the optimizer would be:
```python
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cost)
```

To make the optimization you would do:
```python
_ , c = sess.run([optimizer, cost], feed_dict={X: minibatch_X, Y: minibatch_Y})
```

In [11]:
from tensorflow.python.framework import ops

## 6 - Building the model

In [14]:
def model(learning_rate=0.1,num_epochs=500,minibatch_size=128,print_cost=True):
    ops.reset_default_graph()
    tf.set_random_seed(1)
    seed = 3
    n_x = 784
    n_y = 10
    
    X,Y = create_placeholders(n_x,n_y)
    parameters = initialize_parameters()
    Z3 = forward_propagation(X,parameters)
    cost = compute_cost(Z3,Y)
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    
    correct_prediction = tf.equal(tf.argmax(Z3) , tf.argmax(Y))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction , "float"))
    
    init = tf.global_variables_initializer()
    train_X = mnist.train.images.T
    train_Y = mnist.train.labels.T
    
    with tf.Session() as sess:
        
        sess.run(init)
        for epoch in range(num_epochs):
            seed = seed + 1
            batch_X, batch_Y = mnist.train.next_batch(minibatch_size)
            minibatch_X = batch_X.T
            minibatch_Y = batch_Y.T
            _,minibatch_cost = sess.run([optimizer,cost],feed_dict={X:minibatch_X , Y:minibatch_Y})
            
            
            
            if print_cost and epoch%100 == 0:
                print("cost after epoch %i:%f"%(epoch,minibatch_cost))
                acc = sess.run(accuracy, feed_dict={X: train_X,Y: train_Y})
                print("Accuracy after epoch %i:%f percentage\n"%(epoch,acc*100))
        
        acc = sess.run(accuracy, feed_dict={X: train_X,Y: train_Y})
        print("Accuracy on train images %f\n"%(acc*100))
        acc = sess.run(accuracy, feed_dict={X: mnist.test.images.T,Y: mnist.test.labels.T})
        print("Accuracy on test images %f\n"%(acc*100))
        parameters = sess.run(parameters)
        return parameters

In [15]:
parameters = model()

cost after epoch 0:2.456936
Accuracy after epoch 0:11.749091 percentage

cost after epoch 100:32.844685
Accuracy after epoch 100:83.672726 percentage

cost after epoch 200:9.914660
Accuracy after epoch 200:86.421818 percentage

cost after epoch 300:7.101398
Accuracy after epoch 300:85.141820 percentage

cost after epoch 400:2.654395
Accuracy after epoch 400:87.094545 percentage

Accuracy on train images 88.107270

Accuracy on test images 88.370001

